The previous notebook (inception_resnet_v2_self_trained_on_200x150) created "baseline models" that I can improve on. For example I haven’t used the tabular data and neither have I used any image augmentation. Also, the original architecture likes square images, but I’m giving it rectangular ones. ROI detection net will have to be trained to enable the possibility to detect the main location of the lesion and crop the image accordingly. Both of these points will be addressed next, and this notebook is about improving the results by using tabular data.

In [36]:
import os

import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [37]:
metadata_path = os.path.join('..', 'data', 'HAM10000_metadata.csv')
data = pd.read_csv(metadata_path)
data['age'] = data.groupby('sex')['age'].transform(lambda x: x.fillna(x.mean()))

In [38]:
get_names = lambda root_path: [
    file_name.split('.')[0]
    for dir_path, _, file_names in os.walk(root_path)
    for file_name in file_names
]
train_dir = os.path.join('..', 'data', 'images_original_inception_resnet_v2_200x150_categorized', 'training')
valid_dir = os.path.join('..', 'data', 'images_original_inception_resnet_v2_200x150_categorized', 'validation')
train_names = set(get_names(train_dir))
valid_names = set(get_names(valid_dir))

In [39]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 200),
    batch_size=64,
    class_mode=None) # setting this param to None because the labels will be taken care of separately
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 200),
    batch_size=64,
    class_mode=None)
train_df = data[data['image_id'].isin(train_names)][['dx', 'age', 'sex']]
valid_df = data[data['image_id'].isin(valid_names)][['dx', 'age', 'sex']]

Found 8015 images belonging to 7 classes.
Found 2000 images belonging to 7 classes.


In [40]:
train_age = np.expand_dims(train_df['age'].astype(int).to_numpy(), -1)
train_sex_categories = pd.get_dummies(train_df['sex']).to_numpy()
valid_age = np.expand_dims(valid_df['age'].astype(int).to_numpy(), -1)
valid_sex_categories = pd.get_dummies(valid_df['sex']).to_numpy()
X_train = np.hstack((train_age, train_sex_categories))
y_train = pd.get_dummies(train_df['dx'])
X_valid = np.hstack((valid_age, valid_sex_categories))
y_valid = pd.get_dummies(valid_df['dx'])

The model I build in this notebook will be a multi input one, which means that in order for image data and tabular data play well together, some customizations need to be made: the MultipleInputGenerator class. Sadly, the types were not easy to find, for example I'm not sure what types x_set, y_set are, and so I decided to not use them here at all.

In [41]:
class MultipleInputGenerator(keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.indices = np.arange(self.x[0].shape[0])

    def __len__(self):
        return int(np.ceil(self.x[0].shape[0] / self.batch_size))

    def __getitem__(self, idx):
        indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = [x[indices] for x in self.x]
        batch_y = self.y[indices]

        return batch_x, batch_y

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

In [6]:
SMALLER_WIDTH = 600 // 3
SMALLER_HEIGHT = 450 // 3


def get_model() -> keras.Model:
    base_model = InceptionResNetV2(include_top=False, weights=None, input_shape=(SMALLER_HEIGHT, SMALLER_WIDTH, 3))
    img_x = base_model.output
    img_x = keras.layers.Dropout(.4)(img_x)
    img_x = keras.layers.GlobalAveragePooling2D()(img_x)
    img_x = keras.layers.Dense(512)(img_x)
    img_x = keras.layers.PReLU()(img_x)
    img_x = keras.layers.Dropout(.4)(img_x)
    img_x = keras.layers.Dense(512)(img_x)
    img_x = keras.layers.PReLU()(img_x)
    predictions = keras.layers.Dense(7, activation='softmax')(img_x)
    model = keras.Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def run_model(model_factory, model_name: str) -> None:
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                                   min_delta=1e-6)
    model_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath='models/' + model_name + '{epoch}',
        save_best_only=True)
    tensor_board = keras.callbacks.TensorBoard(log_dir=f'tensor_logs/{model_name}')
    model = model_factory()

    print(model.summary())

    model.fit(
        train_generator,
        validation_data=valid_generator,
        epochs=50,
        callbacks=[early_stopping, model_checkpoint, tensor_board])